In [574]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from sklearn.preprocessing import normalize
from sklearn.linear_model import RidgeCV, LassoCV, LassoLarsCV

In [575]:
import pandas as pd
df_cor = pd.read_csv("data/AmesHousing_selection.csv")

In [576]:
df_cor

,Neighborhood,Overall Qual,Year Built,Garage Area,TotalFullBath,Exter Qual,Kitchen Qual,Gr Liv Area,Foundation,Total Bsmt SF,...,Surface_sup_ext,Bsmt,Saleprice_m2_quartier,1st Flr SF,Full Bath,SalePrice,Garage,Renovation.1,Saleprice_med_quartier,Saleprice_mean_quartier
0,NAmes,6,1960,528.0,2,6,TA,1656,CBlock,1080.0,...,0,22.0,119.55,1656,1,215000,12.0,62,140000.0,145097
1,NAmes,5,1961,730.0,1,6,TA,896,CBlock,882.0,...,120,14.0,119.55,896,1,105000,12.0,61,140000.0,145097
2,NAmes,6,1958,312.0,1,6,Gd,1329,CBlock,1329.0,...,0,14.0,119.55,1329,1,172000,12.0,64,140000.0,145097
3,NAmes,7,1968,522.0,3,8,Ex,2110,CBlock,2110.0,...,0,14.0,119.55,2110,2,244000,12.0,54,140000.0,145097
4,NAmes,6,1970,480.0,2,6,TA,1004,CBlock,1004.0,...,0,14.0,119.55,1004,1,149000,12.0,52,140000.0,145097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,Veenker,6,1977,546.0,2,8,TA,1208,CBlock,1208.0,...,0,16.0,137.72,1208,1,165000,12.0,45,250250.0,248315
2926,Veenker,9,1981,894.0,2,8,Gd,1842,CBlock,1842.0,...,0,24.0,137.72,1842,0,385000,12.0,35,250250.0,248315
2927,GrnHill,7,1998,286.0,2,8,Gd,1502,PConc,1502.0,...,0,16.0,198.66,1502,1,330000,12.0,24,280000.0,280000
2928,GrnHill,7,1986,312.0,2,8,Gd,1295,CBlock,0.0,...,0,14.0,198.66,1295,2,230000,12.0,36,280000.0,280000


In [577]:
q1 = df_cor['Gr Liv Area'].quantile(0.99)

In [578]:
df_cor = df_cor[df_cor['Gr Liv Area'] < q1]

In [579]:
df_cor["Garage"].fillna(12, inplace=True)
df_cor["Total Bsmt SF"].fillna(1052, inplace=True)
df_cor["Garage Area"].fillna(2.0, inplace=True)

/tmp/ipykernel_21963/2868563398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor["Garage"].fillna(12, inplace=True)
/tmp/ipykernel_21963/2868563398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor["Total Bsmt SF"].fillna(1052, inplace=True)
/tmp/ipykernel_21963/2868563398.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor["Garage Area"].fillna(2.0, inplace=True)


# 3. Regularisation

## 3.1. Separattion des données numeriques et categorielles

In [580]:
num_col_cor = []
cat_col_cor = []

for col in df_cor.columns:
    if df_cor[col].dtypes.type in (np.int64, np.float64):
        num_col_cor.append(col)
    else:
        cat_col_cor.append(col)

In [581]:
num_col_cor

['Overall Qual',
 'Year Built',
 'Garage Area',
 'TotalFullBath',
 'Exter Qual',
 'Gr Liv Area',
 'Total Bsmt SF',
 'Total Bath',
 'Overall',
 'Exter',
 'Surface_sup',
 'Renovation',
 'Surface_sup_ext',
 'Bsmt',
 'Saleprice_m2_quartier',
 '1st Flr SF',
 'Full Bath',
 'SalePrice',
 'Garage',
 'Renovation.1',
 'Saleprice_med_quartier',
 'Saleprice_mean_quartier']

In [582]:
cat_col_cor

['Neighborhood', 'Kitchen Qual', 'Foundation']

## 3.2 Selection des 10 meilleures variables 

#Test 1
numeric_features = ['Overall Qual',
                    'Total Bath', 
                    'Total Bsmt SF',
                    'Exter Qual',
                    'SalePrice_m2',
                   'Saleprice_quartier',
                    "Gr Liv Area", 
                    "Garage Area"]

ordinal_features = ['Kitchen Qual']

cat_feature = ["Neighborhood"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)

X = df_cor[all_col]
y = df_cor[["SalePrice"]]

In [583]:
#Test 1
numeric_features = ["Overall",
                    'Total Bsmt SF',
                    'Exter',
                    "Gr Liv Area", 
                    "Garage Area",
                    'Saleprice_m2_quartier',
                    "TotalFullBath",
                    "Bsmt"]

ordinal_features = ['Kitchen Qual']

cat_feature = ["Neighborhood"]

all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)

X = df_cor[all_col]
y = df_cor[["SalePrice"]]

In [584]:
#Test 1
X.columns = ["Overall",
             'Total Bsmt SF',
             'Exter',
             "Gr Liv Area",
             "Garage Area",
             'Saleprice_m2_quartier',
             "TotalFullBath",
             "Bsmt",
             'Kitchen Qual',
             "Neighborhood"]

In [585]:
X.isna().sum()

Overall                  0
Total Bsmt SF            0
Exter                    0
Gr Liv Area              0
Garage Area              0
Saleprice_m2_quartier    0
TotalFullBath            0
Bsmt                     0
Kitchen Qual             0
Neighborhood             0
dtype: int64

In [586]:
#df_cor["Garage"].fillna(12, inplace=True)
df_cor["Total Bsmt SF"].fillna(1052, inplace=True)
df_cor["Garage Area"].fillna(2.0, inplace=True)

/tmp/ipykernel_21963/2848248818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor["Total Bsmt SF"].fillna(1052, inplace=True)
/tmp/ipykernel_21963/2848248818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor["Garage Area"].fillna(2.0, inplace=True)


In [587]:
X.isna().sum()

Overall                  0
Total Bsmt SF            0
Exter                    0
Gr Liv Area              0
Garage Area              0
Saleprice_m2_quartier    0
TotalFullBath            0
Bsmt                     0
Kitchen Qual             0
Neighborhood             0
dtype: int64

In [588]:
X

,Overall,Total Bsmt SF,Exter,Gr Liv Area,Garage Area,Saleprice_m2_quartier,TotalFullBath,Bsmt,Kitchen Qual,Neighborhood
0,11,1080.0,12,1656,528.0,119.55,2,22.0,TA,NAmes
1,11,882.0,12,896,730.0,119.55,1,14.0,TA,NAmes
2,12,1329.0,12,1329,312.0,119.55,1,14.0,Gd,NAmes
3,12,2110.0,14,2110,522.0,119.55,3,14.0,Ex,NAmes
4,13,1004.0,12,1004,480.0,119.55,2,14.0,TA,NAmes
...,...,...,...,...,...,...,...,...,...,...
2925,12,1208.0,14,1208,546.0,137.72,2,16.0,TA,Veenker
2926,14,1842.0,14,1842,894.0,137.72,2,24.0,Gd,Veenker
2927,12,1502.0,14,1502,286.0,198.66,2,16.0,Gd,GrnHill
2928,12,0.0,14,1295,312.0,198.66,2,14.0,Gd,GrnHill


# 3.3. Separation en train & test (eviter le dataleake)

In [590]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [591]:
# Test 1
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder,StandardScaler

numeric_transformer = StandardScaler()

#exter_cat = [ 'Po', 'Fa','TA', 'Gd','Ex']
kitchen_cat = [ 'Po', 'Fa','TA', 'Gd',"Ex"]



ordinal_transformer = OrdinalEncoder(categories=[kitchen_cat])

categorical_transformer = OneHotEncoder()

#Test 1
numeric_features = ['Overall Qual',
                    'Total Bath', 
                    'Total Bsmt SF',
                    'Exter Qual',
                    'SalePrice_m2',
                   'Saleprice_quartier',
                    "Gr Liv Area", 
                    "Garage Area"]

ordinal_features = ['Kitchen Qual']

cat_feature = ["Neighborhood"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)

X = df_cor[all_col]
y = df_cor[["SalePrice"]]m


In [592]:
#Test 1
numeric_features = ["Overall",
                    'Total Bsmt SF',
                    'Exter',
                    "Gr Liv Area", 
                    "Garage Area",
                    'Saleprice_m2_quartier',
                    "TotalFullBath", 
                    "Bsmt",
                   ]

ordinal_features = ['Kitchen Qual']

cat_feature = ["Neighborhood"]

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, cat_feature)
    ]
)

# 3.4 Pipeline & Modele

In [593]:
df_cor


,Neighborhood,Overall Qual,Year Built,Garage Area,TotalFullBath,Exter Qual,Kitchen Qual,Gr Liv Area,Foundation,Total Bsmt SF,...,Surface_sup_ext,Bsmt,Saleprice_m2_quartier,1st Flr SF,Full Bath,SalePrice,Garage,Renovation.1,Saleprice_med_quartier,Saleprice_mean_quartier
0,NAmes,6,1960,528.0,2,6,TA,1656,CBlock,1080.0,...,0,22.0,119.55,1656,1,215000,12.0,62,140000.0,145097
1,NAmes,5,1961,730.0,1,6,TA,896,CBlock,882.0,...,120,14.0,119.55,896,1,105000,12.0,61,140000.0,145097
2,NAmes,6,1958,312.0,1,6,Gd,1329,CBlock,1329.0,...,0,14.0,119.55,1329,1,172000,12.0,64,140000.0,145097
3,NAmes,7,1968,522.0,3,8,Ex,2110,CBlock,2110.0,...,0,14.0,119.55,2110,2,244000,12.0,54,140000.0,145097
4,NAmes,6,1970,480.0,2,6,TA,1004,CBlock,1004.0,...,0,14.0,119.55,1004,1,149000,12.0,52,140000.0,145097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,Veenker,6,1977,546.0,2,8,TA,1208,CBlock,1208.0,...,0,16.0,137.72,1208,1,165000,12.0,45,250250.0,248315
2926,Veenker,9,1981,894.0,2,8,Gd,1842,CBlock,1842.0,...,0,24.0,137.72,1842,0,385000,12.0,35,250250.0,248315
2927,GrnHill,7,1998,286.0,2,8,Gd,1502,PConc,1502.0,...,0,16.0,198.66,1502,1,330000,12.0,24,280000.0,280000
2928,GrnHill,7,1986,312.0,2,8,Gd,1295,CBlock,0.0,...,0,14.0,198.66,1295,2,230000,12.0,36,280000.0,280000


In [594]:
import mlflow

In [595]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [596]:
import numpy as np
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_absolute_error

pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])


pipe.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Overall', 'Total Bsmt SF',
                                                   'Exter', 'Gr Liv Area',
                                                   'Garage Area',
                                                   'Saleprice_m2_quartier',
                                                   'TotalFullBath', 'Bsmt']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex']]),
                                                  ['Kitchen Qual']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Neighborhood'])])),
                ('reg', LinearRegression())])

In [597]:
from sklearn.metrics import mean_absolute_error

with mlflow.start_run():

    pipe.score(X_test,y_test)

    predict_train  = pipe.predict(X_train)
    predict_test  = pipe.predict(X_test)

    # Root Mean Squared Error on train and test date
    mae_train = mean_absolute_error(y_train, predict_train)
    mae_test = mean_absolute_error(y_test, predict_test)
    print('MAE on train data: ', mae_train)
    print('MAE on test data: ',  mean_absolute_error(y_test, predict_test))
    
    mlflow.log_metrics('mae_train', mae_train)
    mlflow.log_metrics('mae_test', mae_test)

MAE on train data:  19151.650982965693
MAE on test data:  16900.99304503446


AttributeError: 'str' object has no attribute 'items'

# 3.5 Enregistrement du modele en fichier joblib

import pickle
filename = 'finalized_model.sav'
pickle.dump(search, open(filename, 'wb'))

In [ ]:
from joblib import dump
dump(pipe, "./model_nounou.joblib")